<a href="https://colab.research.google.com/github/zhiyichenGT/cs7641.github.io/blob/main/NER_release_sp22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Licensing Information:  You are free to use or extend this project for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to The Georgia Institute of Technology, including a link to  https://aritter.github.io/CS-7650-sp22/

# Attribution Information: This assignment was developed at The Georgia Institute of Technology
# by Alan Ritter (alan.ritter@cc.gatech.edu)
# Contributors: Xurui Zhang (Spring 2022)

# Project #2: Named Entity Recognition

In this assignment, you will implement a bidirectional LSTM-CNN-CRF for sequence labeling, following [this paper by Xuezhe Ma and Ed Hovy](https://www.aclweb.org/anthology/P16-1101.pdf), on the CoNLL named entity recognition dataset.  Before starting the assignment, we recommend reading the Ma and Hovy paper.

First, let's import some libraries and make sure the runtime has access to a GPU.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Thu Feb 17 00:54:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download the Data

Run the following code to download the English part of the CoNLL 2003 dataset, the evaluation script and pre-filtered GloVe embeddings we are providing for this data.

In [ ]:
#CoNLL 2003 data
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
!wget https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testb
!cat eng.train | awk '{print $1 "\t" $4}' > train
!cat eng.testa | awk '{print $1 "\t" $4}' > dev
!cat eng.testb | awk '{print $1 "\t" $4}' > test

#Evaluation Script
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl

#Pre-filtered GloVe embeddings
!wget https://raw.githubusercontent.com/aritter/aritter.github.io/master/files/glove.840B.300d.conll_filtered.txt

--2022-02-17 00:55:12--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.06s   

2022-02-17 00:55:12 (48.3 MB/s) - ‘eng.train’ saved [3283420/3283420]

--2022-02-17 00:55:12--  https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

## CoNLL Data Format

Run the following cell to see a sample of the data in CoNLL format.  As you can see, each line in the file represents a word and its labeled named entity tag in BIO format.  A blank line is used to seperate sentences.

In [ ]:
!head -n 20 train

-DOCSTART-	O
	
EU	I-ORG
rejects	O
German	I-MISC
call	O
to	O
boycott	O
British	I-MISC
lamb	O
.	O
	
Peter	I-PER
Blackburn	I-PER
	
BRUSSELS	I-LOC
1996-08-22	O
	
The	O
European	I-ORG


## Reading in the Data

Below we proivide a bit of code to read in data in the CoNLL format.  This also reads in the filtered GloVe embeddings, to save you some effort - we will discuss this more later.

In [ ]:
#Read in the training data
def read_conll_format(filename):
    (words, tags, currentSent, currentTags) = ([],[],['-START-'],['START'])
    for line in open(filename).readlines():
        line = line.strip()
        #print(line)
        if line == "":
            currentSent.append('-END-')
            currentTags.append('END')
            words.append(currentSent)
            tags.append(currentTags)
            (currentSent, currentTags) = (['-START-'], ['START'])
        else:
            (word, tag) = line.split()
            currentSent.append(word)
            currentTags.append(tag)
    return (words, tags)

def sentences2char(sentences):
    return [[['start'] + [c for c in w] + ['end'] for w in l] for l in sentences]


(sentences_train, tags_train) = read_conll_format("train")
(sentences_dev, tags_dev)     = read_conll_format("dev")

print(sentences_train[2])
print(tags_train[2])

sentencesChar = sentences2char(sentences_train)

print(sentencesChar[2])

['-START-', 'Peter', 'Blackburn', '-END-']
['START', 'I-PER', 'I-PER', 'END']
[['start', '-', 'S', 'T', 'A', 'R', 'T', '-', 'end'], ['start', 'P', 'e', 't', 'e', 'r', 'end'], ['start', 'B', 'l', 'a', 'c', 'k', 'b', 'u', 'r', 'n', 'end'], ['start', '-', 'E', 'N', 'D', '-', 'end']]


In [ ]:
#Read GloVe embeddings.
def read_GloVe(filename):
    embeddings = {}
    for line in open(filename).readlines():
        #print(line)
        fields = line.strip().split(" ")
        word = fields[0]
        embeddings[word] = [float(x) for x in fields[1:]]
    return embeddings

GloVe = read_GloVe("glove.840B.300d.conll_filtered.txt")

print(GloVe["the"])
print("dimension of glove embedding:", len(GloVe["the"]))

[0.27204, -0.06203, -0.1884, 0.023225, -0.018158, 0.0067192, -0.13877, 0.17708, 0.17709, 2.5882, -0.35179, -0.17312, 0.43285, -0.10708, 0.15006, -0.19982, -0.19093, 1.1871, -0.16207, -0.23538, 0.003664, -0.19156, -0.085662, 0.039199, -0.066449, -0.04209, -0.19122, 0.011679, -0.37138, 0.21886, 0.0011423, 0.4319, -0.14205, 0.38059, 0.30654, 0.020167, -0.18316, -0.0065186, -0.0080549, -0.12063, 0.027507, 0.29839, -0.22896, -0.22882, 0.14671, -0.076301, -0.1268, -0.0066651, -0.052795, 0.14258, 0.1561, 0.05551, -0.16149, 0.09629, -0.076533, -0.049971, -0.010195, -0.047641, -0.16679, -0.2394, 0.0050141, -0.049175, 0.013338, 0.41923, -0.10104, 0.015111, -0.077706, -0.13471, 0.119, 0.10802, 0.21061, -0.051904, 0.18527, 0.17856, 0.041293, -0.014385, -0.082567, -0.035483, -0.076173, -0.045367, 0.089281, 0.33672, -0.22099, -0.0067275, 0.23983, -0.23147, -0.88592, 0.091297, -0.012123, 0.013233, -0.25799, -0.02972, 0.016754, 0.01369, 0.32377, 0.039546, 0.042114, -0.088243, 0.30318, 0.087747, 0.1634

## Mapping Tokens to Indices

As in the last project, we will need to convert words in the dataset to numeric indices, so they can be presented as input to a neural network.  Code to handle this for you with sample usage is provided below.

In [ ]:
#Create mappings between tokens and indices.

from collections import Counter
import random

#Will need this later to remove 50% of words that only appear once in the training data from the vocabulary (and don't have GloVe embeddings).
wordCounts = Counter([w for l in sentences_train for w in l])
charCounts = Counter([c for l in sentences_train for w in l for c in w])
singletons = set([w for (w,c) in wordCounts.items() if c == 1 and not w in GloVe.keys()])
charSingletons = set([w for (w,c) in charCounts.items() if c == 1])

#Build dictionaries to map from words, characters to indices and vice versa.
#Save first two words in the vocabulary for padding and "UNK" token.
word2i = {w:i+2 for i,w in enumerate(set([w for l in sentences_train for w in l] + list(GloVe.keys())))}
char2i = {w:i+2 for i,w in enumerate(set([c for l in sentencesChar for w in l for c in w]))}
i2word = {i:w for w,i in word2i.items()}
i2char = {i:w for w,i in char2i.items()}

vocab_size = max(word2i.values()) + 1
char_vocab_size = max(char2i.values()) + 1

#Tag dictionaries.
tag2i = {w:i for i,w in enumerate(set([t for l in tags_train for t in l]))}
i2tag = {i:t for t,i in tag2i.items()}

#When training, randomly replace singletons with UNK tokens sometimes to simulate situation at test time.
def getDictionaryRandomUnk(w, dictionary, train=False):
    if train and (w in singletons and random.random() > 0.5):
        return 1
    else:
        return dictionary.get(w, 1)

#Map a list of sentences from words to indices.
def sentences2indices(words, dictionary, train=False):
    #1.0 => UNK
    return [[getDictionaryRandomUnk(w,dictionary, train=train) for w in l] for l in words]

#Map a list of sentences containing to indices (character indices)
def sentences2indicesChar(chars, dictionary):
    #1.0 => UNK
    return [[[dictionary.get(c,1) for c in w] for w in l] for l in chars]

#Indices
X       = sentences2indices(sentences_train, word2i, train=True)
X_char  = sentences2indicesChar(sentencesChar, char2i)
Y       = sentences2indices(tags_train, tag2i)

print("vocab size:", vocab_size)
print("char vocab size:", char_vocab_size)
print()

print("index of word 'the':", word2i["the"])
print("word of index 253:", i2word[253])
print()

#Print out some examples of what the dev inputs will look like
for i in range(10):
    print(" ".join([i2word.get(w,'UNK') for w in X[i]]))

vocab size: 29148
char vocab size: 88

index of word 'the': 23319
word of index 253: Kay

-START- -DOCSTART- -END-
-START- EU rejects German call to boycott British lamb . -END-
-START- Peter Blackburn -END-
-START- BRUSSELS 1996-08-22 -END-
-START- The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep . -END-
-START- Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . -END-
-START- " We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing . -END-
-START- He said further scientific study was required and if it was found that action was needed it should be taken by the European Union . -END-
-

## Padding and Batching

In this assignment, you should train your models using minibatched SGD, rather than using a batch size of 1 as we did in the previous project.  When presenting multiple sentences to the network at the same time, we will need to pad them to be of the same length. We use [torch.nn.utils.rnn.pad_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html) to do so.

Below we provide some code to prepare batches of data to present to the network.  pad the sequence so that all the sequences have the same length.

**Side Note:** PyTorch includes utilities in [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) to help with padding, batching, shuffling and some other things, but for this assignment we will do everything from scratch to help you see exactly how this works.

In [ ]:
#Pad inputs to max sequence length (for batching)
def prepare_input(X_list):
    X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True).type(torch.LongTensor) # padding the sequences with 0
    X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor) # consisting of 0 and 1, 0 for padded positions, 1 for non-padded positions
    return (X_padded, X_mask)

#Maximum word length (for character representations)
MAX_CLEN=32

def prepare_input_char(X_list):
    MAX_SLEN = max([len(l) for l in X_list])
    X_padded  = [l + [[]]*(MAX_SLEN-len(l))  for l in X_list]
    X_padded  = [[w[0:MAX_CLEN] for w in l] for l in X_padded]
    X_padded  = [[w + [1]*(MAX_CLEN-len(w)) for w in l] for l in X_padded]
    return torch.as_tensor(X_padded).type(torch.LongTensor)

#Pad outputs using one-hot encoding
def prepare_output_onehot(Y_list, NUM_TAGS=max(tag2i.values())+1):
    Y_onehot = [torch.zeros(len(l), NUM_TAGS) for l in Y_list]
    for i in range(len(Y_list)):
        for j in range(len(Y_list[i])):
            Y_onehot[i][j,Y_list[i][j]] = 1.0
    Y_padded = torch.nn.utils.rnn.pad_sequence(Y_onehot, batch_first=True).type(torch.FloatTensor)
    return Y_padded

print("max slen:", max([len(x) for x in X_char]))  #Max sequence length in the training data is 39.

(X_padded, X_mask) = prepare_input(X)
X_padded_char      = prepare_input_char(X_char)
Y_onehot           = prepare_output_onehot(Y)

print("X_padded:", X_padded.shape)
print("X_mask:", X_mask.shape)
print("X_padded_char:", X_padded_char.shape)
print("Y_onehot:", Y_onehot.shape)

max slen: 115
X_padded: torch.Size([14987, 115])
X_mask: torch.Size([14987, 115])
X_padded_char: torch.Size([14987, 115, 32])
Y_onehot: torch.Size([14987, 115, 10])


## **Your code starts here:** Basic LSTM Tagger (10 points)

OK, now you should have everything you need to get started.

Recall that your goal is to to implement the BiLSTM-CNN-CRF, as described in [(Ma and Hovy, 2016)](https://www.aclweb.org/anthology/P16-1101.pdf).  This is a relatively complex network with various components.  Below we provide starter code to break down your implementation into increasingly complex versions of the final model, starting with a Basic LSTM tagger.  This way you can be confident that each part is working correctly before incrementally increasing the complexity of your implementation.  This is generally a good approach to take when implementing complex models, since buggy PyTorch code is often partially working, but produces worse results than a correct implementation, so it's hard to know whether added complexities are helping or hurting.  Also, if you aren't able to match published results it's hard to know which component of your model has the problem (or even whether or not it is a problem in the published result!)

Fill in the functions marked as `TODO` in the code block below.  If everything is working correctly, you should be able to achieve an **F1 score of 0.87 on the dev set and 0.83 on the test set (with GloVe embeddings)**. You are required to initialize word embeddings with GloVe later, but you can randomly initialize the word embeddings in the beginning.

In [ ]:
class BasicLSTMtagger(nn.Module):
    def __init__(self, DIM_EMB=10, DIM_HID=10):
        super(BasicLSTMtagger, self).__init__()
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: initialize parameters - embedding layer, nn.LSTM, nn.Linear and nn.LogSoftmax

    def forward(self, X, train=False):
        #TODO: Implement the forward computation.
        return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.

    def init_glove(self, GloVe):
        #TODO: initialize word embeddings using GloVe (you can skip this part in your first version, if you want, see instructions below).
        pass

    def inference(self, sentences):
        X       = prepare_input(sentences2indices(sentences, word2i))[0].cuda()
        pred = self.forward(X).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

    def write_predictions(self, sentences, outFile):
        fOut = open(outFile, 'w')
        for s in sentences:
            y = self.inference([s])[0]
            #print("\n".join(y[1:len(y)-1]))
            fOut.write("\n".join(y[1:len(y)-1]))  #Skip start and end tokens
            fOut.write("\n\n")

#The following code will initialize a model and test that your forward computation runs without errors.
lstm_test   = BasicLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output = lstm_test.forward(prepare_input(X[0:5])[0])
Y_onehot    = prepare_output_onehot(Y[0:5])

#Check the shape of the lstm_output and one-hot label tensors.
print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [ ]:
#Read in the data

(sentences_dev, tags_dev)     = read_conll_format('dev')
(sentences_train, tags_train) = read_conll_format('train')
(sentences_test, tags_test)   = read_conll_format('test')

# Train your Model (10 points)

Next, implement the function below to train your basic BiLSTM tagger.  See [torch.nn.lstm](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).  Make sure to save your predictions on the test set (`test_pred_lstm.txt`) for submission to GradeScope. Feel free to change number of epochs, optimizer, learning rate and batch size.

In [ ]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random

def shuffle_sentences(sentences, tags):
    shuffled_sentences = []
    shuffled_tags      = []
    indices = list(range(len(sentences)))
    random.shuffle(indices)
    for i in indices:
        shuffled_sentences.append(sentences[i])
        shuffled_tags.append(tags[i])
    return (shuffled_sentences, shuffled_tags)

nEpochs = 10

def train_basic_lstm(sentences, tags, lstm):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer

    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0

        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: Impelement gradient update.
            loss = 0
            totalLoss += loss

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

lstm = BasicLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_basic_lstm(sentences_train, tags_train, lstm)

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39476 phrases; correct: 570.
accuracy:   9.89%; precision:   1.44%; recall:   9.59%; FB1:   2.51
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6796
              LOC: precision:   2.71%; recall:  13.66%; FB1:   4.52  9266
             MISC: precision:   1.25%; recall:  12.80%; FB1:   2.28  9429
              ORG: precision:   1.48%; recall:  10.37%; FB1:   2.59  9385
              PER: precision:   1.35%; recall:   3.37%; FB1:   1.92  4600

----------------------------
-START-/START/START Scorers/I-ORG/O :/B-LOC/O Dean/I-LOC/I-PER Saunders/END/I-PER (/END/O 2nd/I-ORG/O minute/START/O ,/O/O 75th/END/O )/I-ORG/O ,/I-PER/O Mark/I-PER/I-PER Hughes/START/I-PER (/O/O 25th/B-ORG/O ,/B-LOC/O 54th/B-MISC/O )/I-PER/O ,/B-ORG/O Andy/O/I-PER Melville/END/I-PER (/B-MISC/O 33rd/I-ORG/O )/B-MISC/O ,/START/O John/B-ORG/I-PER Robinson/I-PER/I-PER (/I-PER/O 45th/B-ORG/O )/END/O ./O/O -END-/I-LOC/END


  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39262 phrases; correct: 622.
accuracy:  10.06%; precision:   1.58%; recall:  10.47%; FB1:   2.75
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6792
              LOC: precision:   3.00%; recall:  15.24%; FB1:   5.02  9329
             MISC: precision:   1.21%; recall:  12.04%; FB1:   2.20  9160
              ORG: precision:   1.78%; recall:  12.38%; FB1:   3.11  9321
              PER: precision:   1.39%; recall:   3.53%; FB1:   2.00  4660



  0%|          | 0/300 [00:00<?, ?it/s]

...

loss on epoch 7 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39361 phrases; correct: 600.
accuracy:  10.31%; precision:   1.52%; recall:  10.10%; FB1:   2.65
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6633
              LOC: precision:   2.95%; recall:  14.81%; FB1:   4.92  9225
             MISC: precision:   1.09%; recall:  10.95%; FB1:   1.98  9281
              ORG: precision:   1.69%; recall:  12.01%; FB1:   2.97  9508
              PER: precision:   1.40%; recall:   3.58%; FB1:   2.01  4714



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 8 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39392 phrases; correct: 579.
accuracy:  10.02%; precision:   1.47%; recall:   9.74%; FB1:   2.55
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6807
              LOC: precision:   2.73%; recall:  13.66%; FB1:   4.55  9187
             MISC: precision:   1.21%; recall:  12.36%; FB1:   2.20  9450
              ORG: precision:   1.66%; recall:  11.48%; FB1:   2.90  9290
              PER: precision:   1.29%; recall:   3.26%; FB1:   1.85  4658



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 9 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39485 phrases; correct: 643.
accuracy:   9.92%; precision:   1.63%; recall:  10.82%; FB1:   2.83
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6685
              LOC: precision:   3.20%; recall:  16.11%; FB1:   5.34  9255
             MISC: precision:   1.28%; recall:  13.02%; FB1:   2.33  9381
              ORG: precision:   1.76%; recall:  12.38%; FB1:   3.08  9440
              PER: precision:   1.29%; recall:   3.31%; FB1:   1.86  4724



In [ ]:
#Evaluation on test data
lstm.write_predictions(sentences_test, 'test_pred_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_lstm.txt | perl conlleval.pl -d "\t"

--2022-02-17 00:55:55--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.1’

conlleval.pl.1      100%[===================>]  12.46K  --.-KB/s    in 0s      

2022-02-17 00:55:55 (42.3 MB/s) - ‘conlleval.pl.1’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 35533 phrases; correct: 565.
accuracy:  10.26%; precision:   1.59%; recall:  10.00%; FB1:   2.74
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6057
              LOC: precision:   2.73%; recall:  13.73%; FB1:   4.55  8397
             MISC: precision:   0.98%; recall:  11.68%; FB1:   1.80  8390
              ORG: preci

## Initialization with GloVe Embeddings (5 points)

If you haven't already, implement the `init_glove()` method in `BasicLSTMtagger` above.

Rather than initializing word embeddings randomly, it is common to use learned word embeddings (GloVe or Word2Vec), as discussed in lecture.  To make this simpler, we have already pre-filtered [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings to only contain words in the vocabulary of the CoNLL NER dataset, and loaded them into a dictionary (`GloVe`) at the beginning of this notebook.



## Character Embeddings (10 points)

Now that you have your basic LSTM tagger working, the next step is to add a convolutional network that computes word embeddings from character representations of words.  See Figure 2 and Figure 3 in the [Ma and Hovy](https://www.aclweb.org/anthology/P16-1101.pdf) paper.  We have provided code in `sentences2input_tensors` to convert sentences into lists of word and character indices.  See also [nn.Conv1d](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html) and [MaxPool1d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html).

Hint: The nn.Conv1d accepts input size $(N, C_{in}, L_{in})$, but we have input size $(N, \text{SLEN}, \text{CLEN}, \text{EMB_DIM})$. We can reshape and [permute](https://pytorch.org/docs/stable/generated/torch.permute.html) our input to satisfy the nn.Conv1d, and recover the dimensions later.

Make sure to save your predictions on the test set, for submission to GradeScope.  You should be able to achieve **90 F1 / 85 F1 on the dev/test sets**.

In [ ]:
import torch.nn.functional as F


class CharLSTMtagger(BasicLSTMtagger): 
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10):
        super(CharLSTMtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID)
        NUM_TAGS = max(tag2i.values())+1

        (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
        #TODO: Initialize parameters.

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.

    def sentences2input_tensors(self, sentences):
        (X, X_mask)   = prepare_input(sentences2indices(sentences, word2i))
        X_char        = prepare_input_char(sentences2indicesChar(sentences, char2i))
        return (X, X_mask, X_char)

    def inference(self, sentences):
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        pred = self.forward(X.cuda(), X_char.cuda()).argmax(dim=2)
        return [[i2tag[pred[i,j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", Y_pred[i])
            print("Gold:\t\t", tags[i])

char_lstm_test = CharLSTMtagger(DIM_HID=7, DIM_EMB=300)
lstm_output    = char_lstm_test.forward(prepare_input(X[0:5])[0], prepare_input_char(X_char[0:5]))
Y_onehot       = prepare_output_onehot(Y[0:5])

print("lstm output shape:", lstm_output.shape)
print("Y onehot shape:", Y_onehot.shape)

lstm output shape: torch.Size([5, 32, 10])
Y onehot shape: torch.Size([5, 32, 10])


In [ ]:
#Training LSTM w/ character embeddings. Feel free to change number of epochs, optimizer, learning rate and batch size.

nEpochs = 10

def train_char_lstm(sentences, tags, lstm):
  #optimizer = optim.Adadelta(lstm.parameters(), lr=0.1)
  #TODO: initialize optimizer

    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0

        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: Gradient update


        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            s = sample(range(len(sentences_dev)), 5)
            lstm.print_predictions([sentences_dev[i] for i in s], [tags_dev[i] for i in s])

char_lstm = CharLSTMtagger(DIM_HID=500, DIM_EMB=300).cuda()
train_char_lstm(sentences_train, tags_train, char_lstm)

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39457 phrases; correct: 587.
accuracy:   9.74%; precision:   1.49%; recall:   9.88%; FB1:   2.59
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6764
              LOC: precision:   2.98%; recall:  15.19%; FB1:   4.99  9348
             MISC: precision:   1.25%; recall:  12.47%; FB1:   2.27  9218
              ORG: precision:   1.49%; recall:  10.51%; FB1:   2.61  9462
              PER: precision:   1.11%; recall:   2.82%; FB1:   1.60  4665

----------------------------
-START-/B-MISC/START I/B-LOC/O think/B-LOC/O the/I-MISC/O reality/START/O is/I-ORG/O that/START/O we/B-LOC/O operate/I-ORG/O in/B-LOC/O emerging/B-MISC/O markets/O/O ,/START/O emerging/B-MISC/O markets/B-LOC/O tend/I-PER/O to/B-MISC/O be/I-LOC/O more/I-MISC/O volatile/END/O ./O/O "/END/O -END-/I-MISC/END
Predicted:	 ['B-MISC', 'B-LOC', 'B-LOC', 'I-MISC', 'START', 'I-ORG', 'START', 'B-LOC', 'I-ORG', 'B-LOC', 'B-MISC', 

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39500 phrases; correct: 612.
accuracy:  10.09%; precision:   1.55%; recall:  10.30%; FB1:   2.69
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6762
              LOC: precision:   2.98%; recall:  15.13%; FB1:   4.98  9338
             MISC: precision:   1.20%; recall:  12.26%; FB1:   2.18  9440
              ORG: precision:   1.67%; recall:  11.63%; FB1:   2.92  9357
              PER: precision:   1.41%; recall:   3.53%; FB1:   2.02  4603



  0%|          | 0/300 [00:00<?, ?it/s]

...

loss on epoch 7 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39414 phrases; correct: 583.
accuracy:  10.09%; precision:   1.48%; recall:   9.81%; FB1:   2.57
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6825
              LOC: precision:   2.81%; recall:  14.37%; FB1:   4.70  9388
             MISC: precision:   1.15%; recall:  11.50%; FB1:   2.09  9234
              ORG: precision:   1.59%; recall:  11.11%; FB1:   2.79  9347
              PER: precision:   1.39%; recall:   3.47%; FB1:   1.98  4620



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 8 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39559 phrases; correct: 628.
accuracy:   9.91%; precision:   1.59%; recall:  10.57%; FB1:   2.76
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6870
              LOC: precision:   3.04%; recall:  15.57%; FB1:   5.09  9405
             MISC: precision:   1.43%; recall:  14.21%; FB1:   2.60  9143
              ORG: precision:   1.65%; recall:  11.56%; FB1:   2.89  9382
              PER: precision:   1.18%; recall:   3.04%; FB1:   1.70  4759



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 9 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39370 phrases; correct: 572.
accuracy:  10.08%; precision:   1.45%; recall:   9.63%; FB1:   2.52
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6752
              LOC: precision:   2.65%; recall:  13.55%; FB1:   4.43  9413
             MISC: precision:   1.23%; recall:  12.47%; FB1:   2.25  9318
              ORG: precision:   1.65%; recall:  11.41%; FB1:   2.88  9269
              PER: precision:   1.19%; recall:   2.99%; FB1:   1.70  4618



In [ ]:
#Evaluation on test set
char_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm.txt | perl conlleval.pl -d "\t"

--2022-02-17 00:56:16--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.2’

conlleval.pl.2      100%[===================>]  12.46K  --.-KB/s    in 0s      

2022-02-17 00:56:16 (38.8 MB/s) - ‘conlleval.pl.2’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 35612 phrases; correct: 567.
accuracy:   9.90%; precision:   1.59%; recall:  10.04%; FB1:   2.75
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6064
              LOC: precision:   3.00%; recall:  15.29%; FB1:   5.02  8489
             MISC: precision:   1.02%; recall:  12.25%; FB1:   1.89  8403
              ORG: preci

## Conditional Random Fields (15 points)

Now we are ready to add a CRF layer to the `CharacterLSTMTagger`.  To train the model, implement `conditional_log_likelihood`, using the score (unnormalized log probability) of the gold sequence, in addition to the partition function, $Z(X)$, which is computed using the forward algorithm.  Then, you can simply use Pytorch's automatic differentiation to compute gradients by running backpropagation through the computation graph of the dynamic program (this should be very simple, so long as you are able to correctly implement the forward algorithm using a computation graph that is supported by PyTorch).  This approach to computing gradients for CRFs is discussed in Section 7.5.3 of the [Eisenstein Book](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)

You will also need to implement the Viterbi algorithm for inference during decoding.

After including CRF training and Viterbi decoding, you should be getting about **92 F1 / 88 F1 on the dev and test set**, respectively.

**IMPORTANT:** Note that training will be substantially slower this time - depending on the efficiency of your implementation, it could take about 5 minutes per epoch (e.g. 50 minutes for 10 iterations).  It is recommended to start out training on a single batch of data (and testing on this same batch), so that you can quickly debug, making sure your model can memorize the labels on a single batch, and then optimize your code.  Once you are fairly confident your code is working properly, then you can train using the full dataset.  We have provided a (commented out) line of code to switch between training on a single batch and the full dataset below.

**Hint #1:** While debugging your implementation of the Forward algorithm it is helpful to look at the loss during training.  The loss should never be less than zero (the log-likelihood should always be negative).

**Hint #2:** To sum log-probabilities in a numerically stable way at the end of the Forward algorithm, you will want to use [`torch.logsumexp`](https://pytorch.org/docs/stable/generated/torch.logsumexp.html).

In [ ]:
#For F.max_pool1d()
import torch.nn.functional as F

class LSTM_CRFtagger(CharLSTMtagger):
    def __init__(self, DIM_EMB=10, DIM_CHAR_EMB=30, DIM_HID=10, N_TAGS=max(tag2i.values())+1):
        super(LSTM_CRFtagger, self).__init__(DIM_EMB=DIM_EMB, DIM_HID=DIM_HID, DIM_CHAR_EMB=DIM_CHAR_EMB)

        #TODO: Initialize parameters.

        self.transitionWeights = nn.Parameter(torch.zeros((N_TAGS, N_TAGS), requires_grad=True))
        nn.init.normal_(self.transitionWeights)

    def gold_score(self, lstm_scores, Y):
        #TODO: compute score of gold sequence Y (unnormalized conditional log-probability)
        return 0

    #Forward algorithm for a single sentence
    #Efficiency will eventually be important here.  We recommend you start by 
    #training on a single batch and make sure your code can memorize the 
    #training data.  Then you can go back and re-write the inner loop using 
    #tensor operations to speed things up.
    def forward_algorithm(self, lstm_scores, sLen):
        #TODO: implement forward algorithm.
        return 0

    def conditional_log_likelihood(self, sentences, tags, train=True):
        #Todo: compute conditional log likelihood of Y (use forward_algorithm and gold_score)
        return 0

    def viterbi(self, lstm_scores, sLen):
        #TODO: Implement Viterbi algorithm, soring backpointers to recover the argmax sequence.  Returns the argmax sequence in addition to its unnormalized conditional log-likelihood.
        return (torch.as_tensor([random.randint(0,lstm_scores.shape[1]-1) for x in range(sLen)]), 0)

    #Computes Viterbi sequences on a batch of data.
    def viterbi_batch(self, sentences):
        viterbiSeqs = []
        (X, X_mask, X_char) = self.sentences2input_tensors(sentences)
        lstm_scores = self.forward(X.cuda(), X_char.cuda())
        for s in range(len(sentences)):
            (viterbiSeq, ll) = self.viterbi(lstm_scores[s], len(sentences[s]))
            viterbiSeqs.append(viterbiSeq)
        return viterbiSeqs

    def forward(self, X, X_char, train=False):
        #TODO: Implement the forward computation.
        return torch.randn((X.shape[0], X.shape[1], self.NUM_TAGS))  #Random baseline.

    def print_predictions(self, words, tags):
        Y_pred = self.inference(words)
        for i in range(len(words)):
            print("----------------------------")
            print(" ".join([f"{words[i][j]}/{Y_pred[i][j]}/{tags[i][j]}" for j in range(len(words[i]))]))
            print("Predicted:\t", [Y_pred[i][j] for j in range(len(words[i]))])
            print("Gold:\t\t", tags[i])

    #Need to use Viterbi this time.
    def inference(self, sentences, viterbi=True):
        pred = self.viterbi_batch(sentences)
        return [[i2tag[pred[i][j].item()] for j in range(len(sentences[i]))] for i in range(len(sentences))]

lstm_crf = LSTM_CRFtagger(DIM_EMB=300).cuda()

In [ ]:
print(lstm_crf.conditional_log_likelihood(sentences_dev[0:3], tags_dev[0:3]))

0


In [ ]:
#CharLSTM-CRF Training. Feel free to change number of epochs, optimizer, learning rate and batch size.
import tqdm
import os
import subprocess
import random

nEpochs = 10

#Get CoNLL evaluation script
os.system('wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl')

def train_crf_lstm(sentences, tags, lstm):
    #optimizer = optim.Adadelta(lstm.parameters(), lr=1.0)
    #TODO: initialize optimizer

    batchSize = 50

    for epoch in range(nEpochs):
        totalLoss = 0.0
        lstm.train()

        #Shuffle the sentences
        (sentences_shuffled, tags_shuffled) = shuffle_sentences(sentences, tags)
        for batch in tqdm.notebook.tqdm(range(0, len(sentences), batchSize), leave=False):
            lstm.zero_grad()
            #TODO: take gradient step on a batch of data.

        print(f"loss on epoch {epoch} = {totalLoss}")
        lstm.write_predictions(sentences_dev, 'dev_pred')   #Performance on dev set
        print('conlleval:')
        print(subprocess.Popen('paste dev dev_pred | perl conlleval.pl -d "\t"', shell=True, stdout=subprocess.PIPE,stderr=subprocess.STDOUT).communicate()[0].decode('UTF-8'))

        if epoch % 10 == 0:
            lstm.eval()
            s = random.sample(range(50), 5)
            lstm.print_predictions([sentences_train[i] for i in s], [tags_train[i] for i in s])   #Print predictions on train data (useful for debugging)

crf_lstm = LSTM_CRFtagger(DIM_HID=500, DIM_EMB=300, DIM_CHAR_EMB=30).cuda()
train_crf_lstm(sentences_train, tags_train, crf_lstm)             #Train on the full dataset
#train_crf_lstm(sentences_train[0:50], tags_train[0:50])          #Train only the first batch (use this during development/debugging)

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 0 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39413 phrases; correct: 576.
accuracy:  10.01%; precision:   1.46%; recall:   9.69%; FB1:   2.54
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6796
              LOC: precision:   2.65%; recall:  13.55%; FB1:   4.43  9411
             MISC: precision:   1.11%; recall:  11.28%; FB1:   2.03  9341
              ORG: precision:   1.66%; recall:  11.41%; FB1:   2.89  9238
              PER: precision:   1.51%; recall:   3.80%; FB1:   2.16  4627

----------------------------
-START-/START/START Spanish/B-ORG/I-MISC Farm/I-MISC/O Minister/I-MISC/O Loyola/B-MISC/I-PER de/B-LOC/I-PER Palacio/I-LOC/I-PER had/O/O earlier/B-MISC/O accused/B-LOC/O Fischler/B-ORG/I-PER at/B-MISC/O an/B-MISC/O EU/START/I-ORG farm/END/O ministers/END/O '/B-LOC/O meeting/I-LOC/O of/B-ORG/O causing/O/O unjustified/I-PER/O alarm/B-ORG/O through/B-MISC/O "/START/O dangerous/END/O generalisation/I-MISC/O ./B-ORG/O "/I-O

  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 1 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39429 phrases; correct: 614.
accuracy:  10.21%; precision:   1.56%; recall:  10.33%; FB1:   2.71
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6799
              LOC: precision:   2.98%; recall:  15.02%; FB1:   4.97  9272
             MISC: precision:   1.24%; recall:  12.58%; FB1:   2.26  9322
              ORG: precision:   1.68%; recall:  11.71%; FB1:   2.95  9320
              PER: precision:   1.38%; recall:   3.53%; FB1:   1.98  4716



  0%|          | 0/300 [00:00<?, ?it/s]

...

loss on epoch 7 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39455 phrases; correct: 577.
accuracy:   9.94%; precision:   1.46%; recall:   9.71%; FB1:   2.54
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6681
              LOC: precision:   2.97%; recall:  15.08%; FB1:   4.97  9312
             MISC: precision:   1.03%; recall:  10.41%; FB1:   1.87  9352
              ORG: precision:   1.54%; recall:  10.74%; FB1:   2.70  9339
              PER: precision:   1.26%; recall:   3.26%; FB1:   1.81  4771



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 8 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39358 phrases; correct: 586.
accuracy:  10.05%; precision:   1.49%; recall:   9.86%; FB1:   2.59
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6718
              LOC: precision:   2.89%; recall:  14.75%; FB1:   4.84  9372
             MISC: precision:   1.08%; recall:  10.85%; FB1:   1.96  9299
              ORG: precision:   1.80%; recall:  12.38%; FB1:   3.14  9231
              PER: precision:   1.03%; recall:   2.66%; FB1:   1.49  4738



  0%|          | 0/300 [00:00<?, ?it/s]

loss on epoch 9 = 0.0
conlleval:
processed 51578 tokens with 5942 phrases; found: 39534 phrases; correct: 612.
accuracy:   9.82%; precision:   1.55%; recall:  10.30%; FB1:   2.69
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  6722
              LOC: precision:   3.08%; recall:  15.62%; FB1:   5.15  9310
             MISC: precision:   1.18%; recall:  12.04%; FB1:   2.16  9373
              ORG: precision:   1.74%; recall:  12.30%; FB1:   3.05  9488
              PER: precision:   1.06%; recall:   2.66%; FB1:   1.51  4641



In [ ]:
crf_lstm.eval()
crf_lstm.write_predictions(sentences_test, 'test_pred_cnn_lstm_crf.txt')
!wget https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
!paste test test_pred_cnn_lstm_crf.txt | perl conlleval.pl -d "\t"

--2022-02-17 00:56:36--  https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/conlleval.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12754 (12K) [text/plain]
Saving to: ‘conlleval.pl.4’

conlleval.pl.4      100%[===================>]  12.46K  --.-KB/s    in 0s      

2022-02-17 00:56:36 (63.9 MB/s) - ‘conlleval.pl.4’ saved [12754/12754]

processed 46666 tokens with 5648 phrases; found: 35757 phrases; correct: 554.
accuracy:   9.74%; precision:   1.55%; recall:   9.81%; FB1:   2.68
                 : precision:   0.00%; recall:   0.00%; FB1:   0.00  5985
              LOC: precision:   2.77%; recall:  14.09%; FB1:   4.63  8483
             MISC: precision:   1.00%; recall:  12.11%; FB1:   1.84  8517
              ORG: preci

## Gradescope

Gradescope allows you to add multiple files to your submission. Please submit this notebook along with the test set prediction:
* test_pred_lstm.txt
* test_pred_cnn_lstm.txt
* test_pred_cnn_lstm_crf.txt
* NER_release.ipynb

To download this notebook, go to `File > Download.ipynb`. You can download the predictions from Colab by clicking the folder icon on the left and finding them under Files. 

Please make sure that you name the files as specified above. You will be able to see the test set accuracy for your predictions. However, the final score will be assigned later based on accuracy and implementation. 

When submitting the .ipynb notebook, please make sure that all the cells run when executed in order starting from a fresh session. If the code doesn't take too long to run, you can re-run everything with `Runtime -> Restart and run all`

You can submit multiple times before the deadline and choose the submission which you want to be graded by going to `Submission History` on gradescope.
